In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_kan
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/transliterated/'

    train_data = read_task(dataset_location , split = 'kan_train_trans')
    dev_data = read_task(dataset_location , split = 'kan_dev_trans')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_kan
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-multilingual-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Kannada' + 'transliterated' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.840865969657898
DEV ACC: 0.5873953349282297
DEV Precision: 0.4478473468799036
DEV Recall: 0.41887270877947774
DEV F1Score: 0.38781120658599555
BEST ACCURACY -->  DEV: 0.5874
BEST PRECISION -->  DEV: 0.44785
BEST RECALL -->  DEV: 0.41887
BEST F1SCORE -->  DEV: 0.38781
TIME PER EPOCH: 4.309296492735545

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.9464807510375977
DEV ACC: 0.6277661483253588
DEV Precision: 0.5071203784976477
DEV Recall: 0.5271

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.8376157879829407
DEV ACC: 0.6078050239234449
DEV Precision: 0.4696424824432846
DEV Recall: 0.5085239784451717
DEV F1Score: 0.467262358357034
BEST ACCURACY -->  DEV: 0.60781
BEST PRECISION -->  DEV: 0.46964
BEST RECALL -->  DEV: 0.50852
BEST F1SCORE -->  DEV: 0.46726
TIME PER EPOCH: 4.490067680676778

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.7677279114723206
DEV ACC: 0.6319527511961722
DEV Precision: 0.4737166024257039
DEV Recall: 0.52677

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.0252403020858765
DEV ACC: 0.6148325358851675
DEV Precision: 0.4718394198608134
DEV Recall: 0.4401281383500635
DEV F1Score: 0.4218266829544215
BEST ACCURACY -->  DEV: 0.61483
BEST PRECISION -->  DEV: 0.47184
BEST RECALL -->  DEV: 0.44013
BEST F1SCORE -->  DEV: 0.42183
TIME PER EPOCH: 4.395302287737528

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.1118062734603882
DEV ACC: 0.6356907894736842
DEV Precision: 0.49855618839364974
DEV Recall: 0.517

FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.138980507850647
DEV ACC: 0.6409988038277512
DEV Precision: 0.5293658994377576
DEV Recall: 0.5221665315366061
DEV F1Score: 0.506478912342403
BEST ACCURACY -->  DEV: 0.65812
BEST PRECISION -->  DEV: 0.54752
BEST RECALL -->  DEV: 0.52217
BEST F1SCORE -->  DEV: 0.50648
TIME PER EPOCH: 4.261780587832133

Training epoch: 7
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.9137371778488159
DEV ACC: 0.6338217703349283
DEV Precision: 0.5672426414090839
DEV Recall: 0.5174682136045772
DEV F1S

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Training epoch: 1
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 1.0374088287353516
DEV ACC: 0.6078050239234449
DEV Precision: 0.4917445435950603
DEV Recall: 0.43857304979631184
DEV F1Score: 0.4264042919627471
BEST ACCURACY -->  DEV: 0.60781
BEST PRECISION -->  DEV: 0.49174
BEST RECALL -->  DEV: 0.43857
BEST F1SCORE -->  DEV: 0.4264
TIME PER EPOCH: 4.346061205863952

Training epoch: 2
FINSIHED BATCH: 19 of 195
FINSIHED BATCH: 39 of 195
FINSIHED BATCH: 59 of 195
FINSIHED BATCH: 79 of 195
FINSIHED BATCH: 99 of 195
FINSIHED BATCH: 119 of 195
FINSIHED BATCH: 139 of 195
FINSIHED BATCH: 159 of 195
FINSIHED BATCH: 179 of 195
Validation loss per 100 evaluation steps: 0.8981660008430481
DEV ACC: 0.6280651913875598
DEV Precision: 0.5146060118753358
DEV Recall: 0.4622

In [6]:
print(best_dev_acc)

0.6640998803827751


In [7]:
print(best_epoch)

8


In [8]:
print(best_dev_precision)

0.5745286242278221


In [9]:
print(best_dev_recall)

0.5444686953366239


In [10]:
print(best_dev_f1score)

0.5262980843243598


In [11]:
print(overall_list_dev_acc)

[[0.5873953349282297, 0.6277661483253588, 0.6503438995215312, 0.6504186602870813, 0.6627541866028708, 0.6324013157894737, 0.6248504784688995, 0.6394288277511961, 0.6309808612440192, 0.6319527511961722], [0.6078050239234449, 0.6319527511961722, 0.6424192583732057, 0.6656698564593302, 0.6489982057416268, 0.6550538277511961, 0.640924043062201, 0.6181967703349283, 0.6386064593301435, 0.6167763157894737], [0.6148325358851675, 0.6356907894736842, 0.6522876794258373, 0.672174043062201, 0.658418062200957, 0.6556519138755981, 0.6627541866028708, 0.6428678229665071, 0.6343450956937798, 0.6636513157894737], [0.5901614832535885, 0.6306070574162679, 0.6248504784688995, 0.658119019138756, 0.6504186602870813, 0.6409988038277512, 0.6338217703349283, 0.6646979665071769, 0.6338217703349283, 0.625299043062201], [0.6078050239234449, 0.6280651913875598, 0.6560257177033493, 0.6451854066985646, 0.6522876794258373, 0.6640998803827751, 0.6494467703349283, 0.6575956937799042, 0.6404754784688995, 0.6380831339712

In [12]:
print(overall_list_dev_precision)

[[0.4478473468799036, 0.5071203784976477, 0.5085689257383952, 0.4928105161327086, 0.5558989605512445, 0.5034852352903999, 0.5201267588848046, 0.5328909522821893, 0.5222887911256896, 0.5250540189645384], [0.4696424824432846, 0.4737166024257039, 0.5024033802298647, 0.5610420515782182, 0.5311646964693686, 0.546028576374059, 0.528501610169391, 0.5325706399879635, 0.54233222577685, 0.5162049762633776], [0.4718394198608134, 0.49855618839364974, 0.5152393351912855, 0.5333774437103527, 0.5455400919638888, 0.5401624091826952, 0.5615520324460717, 0.5399006415840781, 0.5594809734937796, 0.5532076914313756], [0.4635253567408933, 0.48705869516698397, 0.49198817399759043, 0.518514294872654, 0.5475196483331374, 0.5293658994377576, 0.5672426414090839, 0.5968539408797182, 0.541384413620904, 0.5346391690368612], [0.4917445435950603, 0.5146060118753358, 0.516477197123482, 0.5409203953940797, 0.5245014114712258, 0.5396442565464764, 0.5494141700127763, 0.5586750862890746, 0.5745286242278221, 0.551882950240

In [13]:
print(overall_list_dev_recall)

[[0.41887270877947774, 0.5271773946415095, 0.5632388740118123, 0.5383808111080839, 0.5228756760564388, 0.5304389886468411, 0.4693320815516847, 0.5087598944118084, 0.5485554871450593, 0.49981268879664603], [0.5085239784451717, 0.526779383405474, 0.522295409210516, 0.5411567291341426, 0.5283431987688955, 0.5449874123984031, 0.547279705588529, 0.5029084047265866, 0.5234245879679316, 0.49565235566572474], [0.4401281383500635, 0.5170193832459875, 0.5226212520397012, 0.5180324900730192, 0.5660515370813006, 0.5205875845782263, 0.551952066735841, 0.5534763914041988, 0.5243039321828374, 0.5301372295106463], [0.42467047211423525, 0.509066925584657, 0.4936781299079005, 0.5146401392724922, 0.5164425360657418, 0.5221665315366061, 0.5174682136045772, 0.5583938684607133, 0.511135276675665, 0.5566544850839812], [0.43857304979631184, 0.46221637803162213, 0.5351870218314068, 0.539780188983832, 0.5228128403830274, 0.49507344466991243, 0.5406539575991448, 0.5248742163697989, 0.5444686953366239, 0.53969308

In [14]:
print(overall_list_dev_f1score)

[[0.38781120658599555, 0.5013975238810312, 0.5091425510906963, 0.4961090103098499, 0.5134460489102753, 0.494833167966461, 0.47044823067918995, 0.498247915389288, 0.5072987049092569, 0.4861006241822151], [0.467262358357034, 0.4849615826610623, 0.4917897006060991, 0.531063373763631, 0.5095514821239994, 0.52187126162886, 0.5160644389809569, 0.4902688145710744, 0.5127913222986983, 0.48145497498115825], [0.4218266829544215, 0.4875276955855857, 0.49308873344990245, 0.4963275178691621, 0.523505115658888, 0.5123062783830253, 0.5248094950331784, 0.5261276507025046, 0.5172639845088292, 0.5162443938869031], [0.405752767892732, 0.4793345376184233, 0.4560266574432817, 0.5010450544485838, 0.5044064912618406, 0.506478912342403, 0.5062475926984793, 0.5533588741819068, 0.4995676026103467, 0.5205198194326283], [0.4264042919627471, 0.45529742653811806, 0.5029157927464393, 0.512000335034694, 0.5009555908198077, 0.4899576993957952, 0.5252050827269156, 0.5184977114468662, 0.5262980843243598, 0.5127912790586

In [15]:
#The best model is 0